# Report study 34
Registered report: Senescence surveillance of pre-malignant hepatocytes limits liver cancer development

## Import libraries

In [1]:
import pandas as pd
from patsy import dmatrices
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multitest import multipletests

## Protocol 1
Generation of oncogene-induced senescence and immunosurveillance in murine hepatocytes

### [Import data](https://osf.io/cxk87)
In paper it is written that they have taken 5 fields a 200 cells. N=1000 per mouse

In [2]:
def check_comments(row, string):
    if row == row and not (string in row):
        return False
    return True


data = pd.read_csv("data/Protocol_1_Percent_Data.csv")
# data = data[data.apply(lambda row: check_comments(row["Comments"], "Recovered"), axis="columns")][data["Full Injection Received"]=="Y"]
data = data.drop(
    ["Comments", "Full Injection Received", "Mouse", "Cohort"], axis="columns"
)
data["group_1"] = (data["Day"] == 6) & (data["Strain"] == "CB17_WT")
data["group_2"] = (data["Day"] == 30) & (data["Strain"] == "CB17_WT")
data["group_3"] = (data["Day"] == 30) & (data["Strain"] == "CB17_SCID")
data

,Day,Strain,Treatment,P16_percent,P21_percent,NRAS_percent,group_1,group_2,group_3
0,30,CB17_WT,NRAS_G12V,15.406352,4.281647,0.000000,False,True,False
1,30,CB17_SCID,NRAS_G12V,25.158064,0.027420,0.000000,False,False,True
2,30,CB17_SCID,NRAS_G12V,5.597995,0.000000,0.000000,False,False,True
3,30,CB17_SCID,NRAS_G12V_D38A,45.927934,0.018563,0.065268,False,False,True
4,30,CB17_SCID,NRAS_G12V_D38A,7.178571,0.000000,0.000000,False,False,True
5,30,CB17_WT,NRAS_G12V_D38A,34.187192,14.031836,0.715503,False,True,False
6,30,CB17_WT,NRAS_G12V_D38A,65.590343,0.042542,0.000000,False,True,False
7,30,CB17_WT,NRAS_G12V_D38A,25.669739,1.828666,0.723784,False,True,False
8,30,CB17_WT,NRAS_G12V,33.971240,0.024349,0.004640,False,True,False
9,30,CB17_WT,NRAS_G12V,31.948936,0.017553,0.019720,False,True,False


### Analysis

In [3]:
def analysis_function(data, column_name):
    # Three way ANOVA
    formula = f"{column_name} ~ C(Strain) + C(Treatment)+ C(Day)"
    lm = ols(formula, data).fit()
    print("Three-way ANOVA")
    print(lm.summary())
    print(anova_lm(lm))
    # NRAS_G12V
    formula = f"{column_name} ~ C(Strain) + C(Day)"
    lm = ols(formula, data[data["Treatment"] == "NRAS_G12V"]).fit()
    print("NRAS_G12V")
    print(lm.summary())
    anova_table = anova_lm(lm)
    print(anova_table)
    # Bonferroni
    print("Bonferroni")
    print(
        multipletests(anova_table["PR(>F)"].dropna(), alpha=0.025, method="bonferroni")
    )
    formula = f"{column_name} ~ C(group_1)"
    # data["group_1"] = (data["Day"]==6) & (data["Strain"] == "CB17_WT")
    # data["group_2"] = (data["Day"]==30) & (data["Strain"] == "CB17_WT")
    # data["group_3"] = (data["Day"]==30) & (data["Strain"] == "CB17_SCID")
    lm = ols(formula, data[data["group_1"] | data["group_2"]]).fit()
    print("First Bonferroni")
    print(lm.summary())
    print(anova_lm(lm))
    print("Second Bonferroni")
    formula = f"{column_name} ~ C(group_2)"
    # data["group_1"] = (data["Day"]==6) & (data["Strain"] == "CB17_WT")
    # data["group_2"] = (data["Day"]==30) & (data["Strain"] == "CB17_WT")
    # data["group_3"] = (data["Day"]==30) & (data["Strain"] == "CB17_SCID")
    lm = ols(formula, data[data["group_3"] | data["group_2"]]).fit()
    print(lm.summary())
    print(anova_lm(lm))
    # NRAS_G12V_D38A
    formula = f"{column_name} ~ C(Strain) + C(Day)"
    lm = ols(formula, data[data["Treatment"] == "NRAS_G12V_D38A"]).fit()
    print("NRAS_G12V_D38A")
    print(lm.summary())
    print(anova_lm(lm))

In [4]:
def analysis_function(data, column_name):
    # Three way ANOVA
    formula = f"{column_name} ~ C(Strain) + C(Treatment)+ C(Day)"
    lm = ols(formula, data).fit()
    print("Three-way ANOVA")
    print(lm.summary())
    print(anova_lm(lm))
    # NRAS_G12V
    formula = f"{column_name} ~ C(Strain) + C(Day)"
    lm = ols(formula, data[data["Treatment"]=="NRAS_G12V"]).fit()
    print("NRAS_G12V")
    print(lm.summary())
    anova_table = anova_lm(lm)
    print(anova_table)
    # Bonferroni
    print("Bonferroni")
    print(multipletests(anova_table["PR(>F)"].dropna(),alpha=0.025, method="bonferroni"))
    formula = f"{column_name} ~ C(group_1)"
    #data["group_1"] = (data["Day"]==6) & (data["Strain"] == "CB17_WT")
    #data["group_2"] = (data["Day"]==30) & (data["Strain"] == "CB17_WT")
    #data["group_3"] = (data["Day"]==30) & (data["Strain"] == "CB17_SCID")
    lm = ols(formula, data[data["group_1"]|data["group_2"]]).fit()
    print("First Bonferroni")
    print(lm.summary())
    print(anova_lm(lm))
    print("Second Bonferroni")
    formula = f"{column_name} ~ C(group_2)"
    #data["group_1"] = (data["Day"]==6) & (data["Strain"] == "CB17_WT")
    #data["group_2"] = (data["Day"]==30) & (data["Strain"] == "CB17_WT")
    #data["group_3"] = (data["Day"]==30) & (data["Strain"] == "CB17_SCID")
    lm = ols(formula, data[data["group_3"]|data["group_2"]]).fit()
    print(lm.summary())
    print(anova_lm(lm))
    # NRAS_G12V_D38A
    formula = f"{column_name} ~ C(Strain) + C(Day)"
    lm = ols(formula, data[data["Treatment"]=="NRAS_G12V_D38A"]).fit()
    print("NRAS_G12V_D38A")
    print(lm.summary())
    print(anova_lm(lm))

In [5]:
analysis_function(data, "P16_percent")

Three-way ANOVA
                            OLS Regression Results                            
Dep. Variable:            P16_percent   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     1.241
Date:                Fri, 05 May 2023   Prob (F-statistic):              0.308
Time:                        22:01:59   Log-Likelihood:                -201.34
No. Observations:                  44   AIC:                             410.7
Df Residuals:                      40   BIC:                             417.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

In [6]:
analysis_function(data, "P21_percent")

Three-way ANOVA
                            OLS Regression Results                            
Dep. Variable:            P21_percent   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     1.559
Date:                Fri, 05 May 2023   Prob (F-statistic):              0.214
Time:                        22:01:59   Log-Likelihood:                -103.24
No. Observations:                  44   AIC:                             214.5
Df Residuals:                      40   BIC:                             221.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

In [7]:
analysis_function(data, "NRAS_percent")

Three-way ANOVA
                            OLS Regression Results                            
Dep. Variable:           NRAS_percent   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.320
Method:                 Least Squares   F-statistic:                     7.741
Date:                Fri, 05 May 2023   Prob (F-statistic):           0.000342
Time:                        22:01:59   Log-Likelihood:                -15.755
No. Observations:                  44   AIC:                             39.51
Df Residuals:                      40   BIC:                             46.65
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

## Protocol 2
### [Download data](https://osf.io/q586t)

In [8]:
df = pd.read_csv("data/Study_34_Protocol_2.csv")
df = df[df.mouse_id != "ctrl"]  # exclude control
df

,Contents,mouse_id,treatment,strain,cohort,percent_positive,positive_count,negative_count,Area
0,Nras D12 Chrt1 M#57,57,G12V/D38A,BL6WT,1.0,0.01,1,10588,2.482741
1,Nras D12 Chrt1 M#58,58,G12V/D38A,BL6WT,1.0,0.44,57,12766,2.478151
2,Nras D12 Chrt1 M#60,60,G12V/D38A,BL6WT,1.0,0.01,1,9429,2.471604
3,Nras D12 Chrt1 M#62,2,G12V,BL6WT,1.0,0.09,11,12155,2.333911
4,Nras D12 Chrt1 M#63,63,G12V,BL6WT,1.0,0.08,10,12742,2.497275
5,Nras D12 Chrt1 M#72,72,G12V/D38A,CD4,1.0,0.19,17,8755,2.496068
6,Nras D12 Chrt1 M#73,73,G12V/D38A,CD4,1.0,0.19,23,12109,2.462630
7,Nras D12 Chrt1 M#74,74,G12V,CD4,1.0,0.48,43,8994,2.449402
8,Nras D12 Chrt2 M#100,100,G12V,CD4,2.0,0.01,1,12218,2.426555
9,Nras D12 Chrt2 M#101,101,G12V,CD4,2.0,0.05,5,9806,2.462839


In [9]:
# Performing two-way ANOVA
y, X = dmatrices(
    "percent_positive ~ strain + treatment", data=df, return_type="dataframe"
)
model = ols("percent_positive ~ C(treatment) + C(strain)", data=df).fit()
print(model.summary())
anova_table = anova_lm(model, typ=2)
anova_table

                            OLS Regression Results                            
Dep. Variable:       percent_positive   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                 -0.045
Method:                 Least Squares   F-statistic:                    0.5944
Date:                Fri, 05 May 2023   Prob (F-statistic):              0.563
Time:                        22:01:59   Log-Likelihood:                 8.6150
No. Observations:                  20   AIC:                            -11.23
Df Residuals:                      17   BIC:                            -8.243
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

,sum_sq,df,F,PR(>F)
C(treatment),0.02738,1.0,0.940741,0.345686
C(strain),0.00722,1.0,0.248070,0.624819
Residual,0.49478,17.0,NaN,NaN


Planned comparisons with the Bonferroni correction.

a. The percent of Nras-positive cells in wild-type mice injected with NrasG12V compared to the
percent of Nras-positive cells in wild-type mice injected with NrasG12V/D38A.

In [10]:
df_wt = df[df.strain == "BL6WT"]  # exclude control
df_wt

,Contents,mouse_id,treatment,strain,cohort,percent_positive,positive_count,negative_count,Area
0,Nras D12 Chrt1 M#57,57,G12V/D38A,BL6WT,1.0,0.01,1,10588,2.482741
1,Nras D12 Chrt1 M#58,58,G12V/D38A,BL6WT,1.0,0.44,57,12766,2.478151
2,Nras D12 Chrt1 M#60,60,G12V/D38A,BL6WT,1.0,0.01,1,9429,2.471604
3,Nras D12 Chrt1 M#62,2,G12V,BL6WT,1.0,0.09,11,12155,2.333911
4,Nras D12 Chrt1 M#63,63,G12V,BL6WT,1.0,0.08,10,12742,2.497275
14,Nras D12 Chrt2 M#65,65,G12V/D38A,BL6WT,2.0,0.53,53,9935,2.459274
15,Nras D12 Chrt2 M#66,66,G12V/D38A,BL6WT,2.0,0.12,15,12801,2.406333
16,Nras D12 Chrt2 M#67,67,G12V,BL6WT,2.0,0.20,20,10064,2.423465
17,Nras D12 Chrt2 M#68,68,G12V,BL6WT,2.0,0.01,1,7607,2.337331
18,Nras D12 Chrt2 M#70,70,G12V,BL6WT,2.0,0.01,1,9294,2.433537


In [11]:
# Performing one-way ANOVA
model_a = ols("percent_positive ~ C(treatment)", data=df).fit()
print(model_a.summary())
anova_table_a = anova_lm(model_a, typ=1)
anova_table_a

                            OLS Regression Results                            
Dep. Variable:       percent_positive   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9818
Date:                Fri, 05 May 2023   Prob (F-statistic):              0.335
Time:                        22:01:59   Log-Likelihood:                 8.4701
No. Observations:                  20   AIC:                            -12.94
Df Residuals:                      18   BIC:                            -10.95
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

,df,sum_sq,mean_sq,F,PR(>F)
C(treatment),1.0,0.02738,0.027380,0.981753,0.3349
Residual,18.0,0.50200,0.027889,NaN,NaN


In [12]:
anova_lm(model_a, typ=2)

,sum_sq,df,F,PR(>F)
C(treatment),0.02738,1.0,0.981753,0.3349
Residual,0.50200,18.0,NaN,NaN


In [13]:
# With Bonferroni correction with standard alpha from statsmodels
multipletests(
    [0.3349], alpha=0.05, method="bonferroni", is_sorted=False, returnsorted=False
)

(array([False]), array([0.3349]), 0.050000000000000044, 0.05)

b. The percent of Nras-positive cells in wild-type mice injected with NrasG12V compared to the
percent of Nras-positive cells in CD4−/− mice injected with NrasG12V.

In [14]:
# get df treatment NrasG12V
df_g12v = df[df.treatment == "G12V"]  # exclude control
df_g12v

,Contents,mouse_id,treatment,strain,cohort,percent_positive,positive_count,negative_count,Area
3,Nras D12 Chrt1 M#62,2,G12V,BL6WT,1.0,0.09,11,12155,2.333911
4,Nras D12 Chrt1 M#63,63,G12V,BL6WT,1.0,0.08,10,12742,2.497275
7,Nras D12 Chrt1 M#74,74,G12V,CD4,1.0,0.48,43,8994,2.449402
8,Nras D12 Chrt2 M#100,100,G12V,CD4,2.0,0.01,1,12218,2.426555
9,Nras D12 Chrt2 M#101,101,G12V,CD4,2.0,0.05,5,9806,2.462839
16,Nras D12 Chrt2 M#67,67,G12V,BL6WT,2.0,0.20,20,10064,2.423465
17,Nras D12 Chrt2 M#68,68,G12V,BL6WT,2.0,0.01,1,7607,2.337331
18,Nras D12 Chrt2 M#70,70,G12V,BL6WT,2.0,0.01,1,9294,2.433537
20,Nras D12 Chrt2 M#97,97,G12V,CD4,2.0,0.00,0,10332,2.378261
21,Nras D12 Chrt2 M#98,98,G12V,CD4,2.0,0.01,1,12037,2.464747


In [15]:
# Performing one-way ANOVA
model_b = ols("percent_positive ~ C(strain)", data=df).fit()
print(model_b.summary())
anova_table_b = anova_lm(model_b, typ=1)
anova_table_b

                            OLS Regression Results                            
Dep. Variable:       percent_positive   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.041
Method:                 Least Squares   F-statistic:                    0.2489
Date:                Fri, 05 May 2023   Prob (F-statistic):              0.624
Time:                        22:01:59   Log-Likelihood:                 8.0764
No. Observations:                  20   AIC:                            -12.15
Df Residuals:                      18   BIC:                            -10.16
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1500      0.054  

,df,sum_sq,mean_sq,F,PR(>F)
C(strain),1.0,0.00722,0.007220,0.248889,0.6239
Residual,18.0,0.52216,0.029009,NaN,NaN


In [16]:
anova_lm(model_b, typ=2)

,sum_sq,df,F,PR(>F)
C(strain),0.00722,1.0,0.248889,0.6239
Residual,0.52216,18.0,NaN,NaN


In [17]:
# With Bonferroni correction with standard alpha from statsmodels
multipletests(
    [0.6239], alpha=0.05, method="bonferroni", is_sorted=False, returnsorted=False
)

(array([False]), array([0.6239]), 0.050000000000000044, 0.05)

### Environment

In [18]:
with open("../environment.yml", "r") as f:
    content = f.read()
print(content)

name: reproducibility_hackathon
channels:
  - conda-forge
  - defaults
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=2_gnu
  - bzip2=1.0.8=h7f98852_4
  - ca-certificates=2022.12.7=ha878542_0
  - ld_impl_linux-64=2.40=h41732ed_0
  - libexpat=2.5.0=hcb278e6_1
  - libffi=3.4.2=h7f98852_5
  - libgcc-ng=12.2.0=h65d4601_19
  - libgomp=12.2.0=h65d4601_19
  - libnsl=2.0.0=h7f98852_0
  - libsqlite=3.40.0=h753d276_1
  - libuuid=2.38.1=h0b41bf4_0
  - libzlib=1.2.13=h166bdaf_4
  - ncurses=6.3=h27087fc_1
  - openssl=3.1.0=hd590300_3
  - pip=23.1.2=pyhd8ed1ab_0
  - python=3.11.3=h2755cc3_0_cpython
  - readline=8.2=h8228510_1
  - setuptools=67.7.2=pyhd8ed1ab_0
  - tk=8.6.12=h27826a3_0
  - wheel=0.40.0=pyhd8ed1ab_0
  - xz=5.2.6=h166bdaf_0
  - pip:
      - anyio==3.6.2
      - argon2-cffi==21.3.0
      - argon2-cffi-bindings==21.2.0
      - arrow==1.2.3
      - asttokens==2.2.1
      - attrs==23.1.0
      - backcall==0.2.0
      - beautifulsoup4==4.12.2
      - bleach==6.0.0
   